In [ ]:
!pip install pandas openpyxl


In [ ]:
!pip install fair-esm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 2.8 MB/s eta 0:00:00


In [ ]:
import torch
import esm
from sklearn.cluster import KMeans

import os
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/Masters Thesis/unlabeled_data.csv'

# data loading and pre-processing
data = pd.read_csv(file_path)
data1 = data["Sequence"]
data1

0       NADNYKNVINRTGAPQYMKDYDYDDHQRFNPFFDLGAWHGHLLPDG...
1       TKSVKRGVAYDVASPADLSALSTGMSWWYNWSPKPHDRLAAYDYAG...
2       MSTSSISLSLMAALMLSAGLLLGCSEKPAESAAAVADSATTTAPQS...
3       GAPSNIAGMIVFLDPGHNGANDASIGRQVPTGRGGTKNCQESGTAT...
4       TEKRGRIYLKAEVTDEKLHVTVRDAKNLIPMDPNGLSDPYVKLKLI...
                              ...                        
3223    MTESYDVVVVGGGPVGLATAWQVAERGHRVLVLERHTFFNENGGTS...
3224    SNAMTGGRFDFDDGGTYCGGWEEGKAHGHGICTGPKGQGEYSGSWS...
3225    GDYNQTVLSHLQKFWKHHDIKGFTWTLGRIVEELPDFQVFQVIPNH...
3226    MGSSHHHHHHSSGLVPRGSHMASGIARGRLAEERKSWRKNHPHGFV...
3227    MSVGLIAPLASPIQESRANTNIENIGDGAEVIKRTEDVSSKKWGVT...
Name: Sequence, Length: 3228, dtype: object

ESM2 model download

In [ ]:
model, alphabet = esm.pretrained.esm2_t36_3B_UR50D()

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t36_3B_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t36_3B_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t36_3B_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t36_3B_UR50D-contact-regression.pt


Extracting embeddings for unlabeled_data

In [ ]:
# Define the directory  to save the embeddings

output_dir = '/content/drive/My Drive/Masters Thesis/esm2_unlabeled'

# Check if the directory exists, if not create it
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Set the device to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move  model to the chosen device
model.to(device)

# data processing loop
for index, seq in enumerate(data['Sequence'].values):
    try:
        # Encode the sequence
        encoded_sequence = torch.tensor(alphabet.encode(seq)).unsqueeze(0)

        # Move encoded sequence to the same device as the model
        encoded_sequence = encoded_sequence.to(device)

        with torch.no_grad():
            # Extract embeddings from the required layer
            results = model(encoded_sequence, repr_layers=[36])
            embeddings = results['representations'][36].mean(0)

            # The filename for the embedding
            output_file_path = os.path.join(output_dir, f"embedding_{index}.npy")

            # Save embeddings as numpy file, ensure they are on cpu
            np.save(output_file_path, embeddings.cpu().numpy())

    except Exception as e:
        print(f"An error occurred at index {index}: {e}")



In [ ]:
#taking the mean and save as csv file
folder_path = '/content/drive/My Drive/Masters Thesis/esm2_unlabeled'
files = [f for f in os.listdir(folder_path) if f.endswith('.npy')]

data_list = []

for file in files:
    path = os.path.join(folder_path, file)
    datt = np.load(path)
    averaged_datt = datt.mean(axis=0, keepdims=True)
    data_list.append(averaged_datt.squeeze())

esm2_unlabeled = pd.DataFrame(data_list)
esm2_unlabeled.to_csv('/content/drive/My Drive/Masters Thesis/esm2_unlabeled.csv', index=False)
esm2_unlabeled.head()

,0,1,2,3,4,5,6,7,8,9,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,-0.146392,-0.044388,-0.073572,-0.083538,0.013867,-0.115059,-0.021020,0.251900,0.082191,-0.110854,...,0.104507,-0.209106,0.026390,-0.034675,0.164679,-0.046357,-0.086354,0.092409,-0.050137,-0.178787
1,-0.097094,0.003643,-0.010534,-0.083778,0.009176,-0.063397,-0.049395,0.153481,0.013189,-0.057253,...,0.057338,-0.095117,0.106728,-0.085964,0.067033,0.001800,0.046425,0.032737,-0.098372,-0.114315
2,0.005237,-0.020602,-0.002926,-0.080908,0.027083,0.034999,-0.044002,0.044819,0.000701,-0.027679,...,0.055709,-0.095342,0.042804,-0.025231,0.053732,-0.037928,0.064099,0.038445,-0.057057,-0.022486
3,-0.012934,-0.014526,-0.105428,-0.010334,-0.017943,-0.113075,0.023310,0.148012,0.044867,-0.084258,...,0.115832,-0.082564,0.045533,-0.065703,0.012751,-0.011484,0.043135,0.027399,-0.050511,-0.100954
4,0.027225,-0.053654,-0.052486,-0.090339,0.027644,-0.001249,-0.029163,0.170268,0.087543,-0.038012,...,0.063100,-0.113716,0.046959,-0.027659,0.130872,-0.057488,0.032790,0.111607,-0.140744,-0.089814


Extract the embedding for labeled data sequence

1.   Listepunkt
2.   Listepunkt



In [ ]:
file_path = '/content/drive/My Drive/Masters Thesis/labeled_data.csv'

# data loading and pre-processing
data2 = pd.read_csv(file_path)
data2 = data["Sequence"]
data2

Extract embedding

In [ ]:
# Define the directory  to save the embeddings

output_dir = '/content/drive/My Drive/Masters Thesis/esm2_labeled'

# Check if the directory exists, if not create it
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Set the device to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move  model to the chosen device
model.to(device)

# data processing loop
for index, seq in enumerate(data2['Sequence'].values):
    try:
        # Encode the sequence
        encoded_sequence = torch.tensor(alphabet.encode(seq)).unsqueeze(0)

        # Move encoded sequence to the same device as the model
        encoded_sequence = encoded_sequence.to(device)

        with torch.no_grad():
            # Extract embeddings from the required layer
            results = model(encoded_sequence, repr_layers=[36])
            embeddings = results['representations'][36].mean(0)

            # The filename for the embedding
            output_file_path = os.path.join(output_dir, f"embedding_{index}.npy")

            # Save embeddings as numpy file, ensure they are on cpu
            np.save(output_file_path, embeddings.cpu().numpy())

    except Exception as e:
        print(f"An error occurred at index {index}: {e}")




In [ ]:
#taking the mean and save as csv file
folder_path = '/content/drive/My Drive/Masters Thesis/esm2_labeled'
files = [f for f in os.listdir(folder_path) if f.endswith('.npy')]

data_list = []

for file in files:
    path = os.path.join(folder_path, file)
    datt = np.load(path)
    averaged_datt = datt.mean(axis=0, keepdims=True)
    data_list.append(averaged_datt.squeeze())

esm2_labeled = pd.DataFrame(data_list)
esm2_labeled.to_csv('/content/drive/My Drive/Masters Thesis/esm2_labeled.csv', index=False)
esm2_labeled.head()